## Data acquisition

In [21]:
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
  
# Fetching the dataset. 
student_performance = fetch_ucirepo(id=320) 

categorical_columns = student_performance.variables[student_performance.variables['type'].isin(['Categorical', 'Binary'])]

# Some of the features are categorical - we'll need to encode them later on, we extract the indices of columns already.
indices = categorical_columns.index.tolist()
indices = [index for index in indices if index < 30]
   
X = student_performance.data.features
y = student_performance.data.targets

# Extracting the targets
G1_multiclass = y.iloc[:, 0]
G2_multiclass = y.iloc[:, 1]
G3_multiclass = y.iloc[:, 2]

# Move G1 and G2 to features for later use.
X_with_G1 = pd.concat([X, G1_multiclass], axis=1)
X_with_G1_G2 = pd.concat([X_with_G1, G2_multiclass], axis=1)

# Student's perfomance is in a 1-20 scale, we'll turn it into a binary classification problem - passing the class or failing it.
threshold = 10
G1 = np.where(G1_multiclass >= threshold, 1, 0)
G2 = np.where(G2_multiclass >= threshold, 1, 0)
G3 = np.where(G3_multiclass >= threshold, 1, 0)

# Although we will use the same data for all three groups, we need to get the same splits for each group.
G1_X_train, G1_X_test, G1_y_train, G1_y_test = train_test_split(X, G1, test_size=0.2, random_state=2137)
G2_X_train, G2_X_test, G2_y_train, G2_y_test = train_test_split(X, G2, test_size=0.2, random_state=2137)
G3_X_train, G3_X_test, G3_y_train, G3_y_test = train_test_split(X, G3, test_size=0.2, random_state=2137)
G3_X_with_G1_G2_train, G3_X_with_G1_G2_test, G3_y_with_G1_G2_train, G3_y_with_G1_G2_test = train_test_split(X_with_G1_G2, G3, test_size=0.2, random_state=2137)

## Data preprocessing and AdaBoost model preparation


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

model = AdaBoostClassifier()

# Defining the preprocessing steps. We will use them for both models.
preprocessing_steps = [
  # We need to convert the categorical features.
  ('encoder', ColumnTransformer(transformers=[('encoder', OrdinalEncoder(), indices)], remainder='passthrough')),
  # Normalizing the data.
  ('scaler', StandardScaler()),
]

qualification_steps = [
    ('model', AdaBoostClassifier(estimator=DecisionTreeClassifier(), random_state=2137, algorithm='SAMME')),
]

# Creating the pipeline for AdaBoostClassifier.
pipeline = Pipeline(steps=preprocessing_steps + qualification_steps)

## Find the best hyperparameters for AdaBoost model with RandomizedSearchCV

In [34]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'model__n_estimators': np.arange(4, 128, 4),
    'model__learning_rate': np.arange(0.05, 1.0, 0.05),
    'model__estimator__max_depth': np.arange(1, 10, 1),
    'model__estimator__min_samples_leaf': np.arange(1, 10, 1),
}

# Create a RandomizedSearchCV object with the pipeline.
grid_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, n_iter=1024, scoring='accuracy', n_jobs=-1, verbose=1, random_state=2137) 

# Fit the data to the GridSearchCV object.
bestModel = grid_search.fit(G1_X_train, G1_y_train).best_estimator_

# Get the best parameters and the best score.
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Make the predictions.
G1_pred = bestModel.predict(G1_X_test)
G2_pred = bestModel.predict(G2_X_test)
G3_pred = bestModel.predict(G3_X_test)

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
Best Parameters: {'model__n_estimators': 24, 'model__learning_rate': 0.6500000000000001, 'model__estimator__min_samples_leaf': 7, 'model__estimator__max_depth': 2}
Best Score: 0.8343726661687827


## AdaBoosts model's metrics

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

accuracy_G1 = accuracy_score(y_true=G1_y_test, y_pred=G1_pred)
accuracy_G2 = accuracy_score(y_true=G2_y_test, y_pred=G2_pred)
accuracy_G3 = accuracy_score(y_true=G3_y_test, y_pred=G3_pred)
print("Accuracy on G1:", accuracy_G1)
print("Accuracy on G2:", accuracy_G2)
print("Accuracy on G3:", accuracy_G3)

precision_G1 = precision_score(y_true=G1_y_test, y_pred=G1_pred)
precision_G2 = precision_score(y_true=G2_y_test, y_pred=G2_pred)
precision_G3 = precision_score(y_true=G3_y_test, y_pred=G3_pred)
print("Precision on G1:", precision_G1)
print("Precision on G2:", precision_G2)
print("Precision on G3:", precision_G3)

recall_G1 = recall_score(y_true=G1_y_test, y_pred=G1_pred)
recall_G2 = recall_score(y_true=G2_y_test, y_pred=G2_pred)
recall_G3 = recall_score(y_true=G3_y_test, y_pred=G3_pred)
print("Recall on G1:", recall_G1)
print("Recall on G2:", recall_G2)
print("Recall on G3:", recall_G3)

f1_G1 = f1_score(y_true=G1_y_test, y_pred=G1_pred)
f1_G2 = f1_score(y_true=G2_y_test, y_pred=G2_pred)
f1_G3 = f1_score(y_true=G3_y_test, y_pred=G3_pred)
print("F1 Score on G1:", f1_G1)
print("F1 Score on G2:", f1_G2)
print("F1 Score on G3:", f1_G3)

conf_matrix_G1 = confusion_matrix(y_true=G1_y_test, y_pred=G1_pred)
conf_matrix_G2 = confusion_matrix(y_true=G2_y_test, y_pred=G2_pred)
conf_matrix_G3 = confusion_matrix(y_true=G3_y_test, y_pred=G3_pred)
print("Confusion Matrix on G1:\n", conf_matrix_G1)
print("Confusion Matrix on G2:\n", conf_matrix_G2)
print("Confusion Matrix on G3:\n", conf_matrix_G3)

target_names = ['Fail', 'Pass']
classification_report_G1 = classification_report(y_true=G1_y_test, y_pred=G1_pred, target_names=target_names)
classification_report_G2 = classification_report(y_true=G2_y_test, y_pred=G2_pred, target_names=target_names)
classification_report_G3 = classification_report(y_true=G3_y_test, y_pred=G3_pred, target_names=target_names)
print("Classification Report on G1:\n", classification_report_G1)
print("Classification Report on G2:\n", classification_report_G2)
print("Classification Report on G3:\n", classification_report_G3) 

Accuracy on G1: 0.8076923076923077
Accuracy on G2: 0.7846153846153846
Accuracy on G3: 0.8307692307692308
Precision on G1: 0.8365384615384616
Precision on G2: 0.8269230769230769
Precision on G3: 0.9038461538461539
Recall on G1: 0.9157894736842105
Recall on G2: 0.8958333333333334
Recall on G3: 0.8867924528301887
F1 Score on G1: 0.8743718592964824
F1 Score on G2: 0.86
F1 Score on G3: 0.8952380952380953
Confusion Matrix on G1:
 [[18 17]
 [ 8 87]]
Confusion Matrix on G2:
 [[16 18]
 [10 86]]
Confusion Matrix on G3:
 [[14 10]
 [12 94]]
Classification Report on G1:
               precision    recall  f1-score   support

        Fail       0.69      0.51      0.59        35
        Pass       0.84      0.92      0.87        95

    accuracy                           0.81       130
   macro avg       0.76      0.72      0.73       130
weighted avg       0.80      0.81      0.80       130

Classification Report on G2:
               precision    recall  f1-score   support

        Fail       0.62

## Neural network approach


In [25]:
from keras_tuner import RandomSearch
from keras import models, layers, callbacks

# We'll use keras_tuner to search for the best hyperparameters.
def build_model(hp):
    model = models.Sequential([
        layers.Input((30,)),
    ])

    # We want to find the best starting size for the hidden layers.
    units = hp.Int('units', min_value=32, max_value=128, step=32)
    current_units = units

    # We want to find the best number of hidden layers.
    for i in range(1, hp.Int('hidden_layers', min_value=2, max_value=6, step=1)):

        # We want to find the best activation function for everu hidden layer.
        model.add(layers.Dense(units=current_units, activation=hp.Choice('activation', ['relu', 'sigmoid', 'softmax'])))
        # Batch normalization is a good idea.
        model.add(layers.BatchNormalization())
        # We want to find the best dropout rate for every hidden layer.
        model.add(layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
        # We want to find the best ratio of input to output units for every hidden layer.
        current_units = int(current_units // hp.Float('size_factor', min_value=1.0, max_value=4.0, step=0.5))

    # Since the problem is binary, we want to use a sigmoid activation function.
    model.add(layers.Dense(1, activation='sigmoid',))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

early_stopping = callbacks.EarlyStopping(monitor='accuracy', patience=8) 

# We need to get the data preprocessed before we can use it for training.
transformed_G1_X_train = Pipeline(steps=preprocessing_steps).fit_transform(G1_X_train, G1_y_train)
transformed_G1_X_test = Pipeline(steps=preprocessing_steps).fit_transform(G1_X_test, G1_y_test)

transformed_G2_X_test = Pipeline(steps=preprocessing_steps).fit_transform(G2_X_test, G2_y_test)
transformed_G3_X_test = Pipeline(steps=preprocessing_steps).fit_transform(G3_X_test, G3_y_test)

# Since there's numerous combinations of hyperparameters, we'll use a random search to find the best combination.
tuner = RandomSearch(hypermodel=build_model, objective='val_accuracy', max_trials=128, seed=2137, project_name='G1')
tuner.search(transformed_G1_X_train, G1_y_train, epochs=128, callbacks=[early_stopping], validation_data=(transformed_G1_X_test, G1_y_test), verbose=1)

model = tuner.get_best_models()[0]
print(tuner.get_best_hyperparameters()[0].values)

G1_pred_train = model.predict(transformed_G1_X_train)
G1_pred_train = np.where(G1_pred_train >= 0.5, 1, 0)

accuracy_G3 = accuracy_score(y_true=G1_y_train, y_pred=G1_pred_train)
print("Accuracy on train G1:", accuracy_G1)

G1_pred = model.predict(transformed_G1_X_test)
G1_pred = np.where(G1_pred >= 0.5, 1, 0)

G2_pred = model.predict(transformed_G2_X_test)
G2_pred = np.where(G2_pred >= 0.5, 1, 0)

G3_pred = model.predict(transformed_G3_X_test)
G3_pred = np.where(G3_pred >= 0.5, 1, 0)

Reloading Tuner from ./G1/tuner0.json
{'units': 64, 'hidden_layers': 6, 'activation': 'relu', 'dropout': 0.2, 'size_factor': 1.5}
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Accuracy on train G1: 0.8076923076923077
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step


/Users/bigpoppe/UJ/PSI/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step


# Neural network metrics

In [26]:
print(model.summary())
accuracy_G1 = accuracy_score(y_true=G1_y_test, y_pred=G1_pred)
accuracy_G2 = accuracy_score(y_true=G2_y_test, y_pred=G2_pred)
accuracy_G3 = accuracy_score(y_true=G3_y_test, y_pred=G3_pred)
print("Accuracy on G1:", accuracy_G1)
print("Accuracy on G2:", accuracy_G2)
print("Accuracy on G3:", accuracy_G3)

precision_G1 = precision_score(y_true=G1_y_test, y_pred=G1_pred)
precision_G2 = precision_score(y_true=G2_y_test, y_pred=G2_pred)
precision_G3 = precision_score(y_true=G3_y_test, y_pred=G3_pred)
print("Precision on G1:", precision_G1)
print("Precision on G2:", precision_G2)
print("Precision on G3:", precision_G3)

recall_G1 = recall_score(y_true=G1_y_test, y_pred=G1_pred)
recall_G2 = recall_score(y_true=G2_y_test, y_pred=G2_pred)
recall_G3 = recall_score(y_true=G3_y_test, y_pred=G3_pred)
print("Recall on G1:", recall_G1)
print("Recall on G2:", recall_G2)
print("Recall on G3:", recall_G3)

f1_G1 = f1_score(y_true=G1_y_test, y_pred=G1_pred)
f1_G2 = f1_score(y_true=G2_y_test, y_pred=G2_pred)
f1_G3 = f1_score(y_true=G3_y_test, y_pred=G3_pred)
print("F1 Score on G1:", f1_G1)
print("F1 Score on G2:", f1_G2)
print("F1 Score on G3:", f1_G3)

conf_matrix_G1 = confusion_matrix(y_true=G1_y_test, y_pred=G1_pred)
conf_matrix_G2 = confusion_matrix(y_true=G2_y_test, y_pred=G2_pred)
conf_matrix_G3 = confusion_matrix(y_true=G3_y_test, y_pred=G3_pred)
print("Confusion Matrix on G1:\n", conf_matrix_G1)
print("Confusion Matrix on G2:\n", conf_matrix_G2)
print("Confusion Matrix on G3:\n", conf_matrix_G3)

target_names = ['Fail', 'Pass']
classification_report_G1 = classification_report(y_true=G1_y_test, y_pred=G1_pred, target_names=target_names)
classification_report_G2 = classification_report(y_true=G2_y_test, y_pred=G2_pred, target_names=target_names)
classification_report_G3 = classification_report(y_true=G3_y_test, y_pred=G3_pred, target_names=target_names)
print("Classification Report on G1:\n", classification_report_G1)
print("Classification Report on G2:\n", classification_report_G2)
print("Classification Report on G3:\n", classification_report_G3) 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 42)             │         2,730 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 42)             │           168 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 28)             │         1,204 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28)             │           112 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 18)             │           522 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 18)             │            72 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 18)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 12)             │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12)             │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,337 (28.66 KB)

 Trainable params: 7,009 (27.38 KB)

 Non-trainable params: 328 (1.28 KB)

None
Accuracy on G1: 0.8538461538461538
Accuracy on G2: 0.8153846153846154
Accuracy on G3: 0.8461538461538461
Precision on G1: 0.88
Precision on G2: 0.86
Precision on G3: 0.93
Recall on G1: 0.9263157894736842
Recall on G2: 0.8958333333333334
Recall on G3: 0.8773584905660378
F1 Score on G1: 0.9025641025641026
F1 Score on G2: 0.8775510204081632
F1 Score on G3: 0.9029126213592233
Confusion Matrix on G1:
 [[23 12]
 [ 7 88]]
Confusion Matrix on G2:
 [[20 14]
 [10 86]]
Confusion Matrix on G3:
 [[17  7]
 [13 93]]
Classification Report on G1:
               precision    recall  f1-score   support

        Fail       0.77      0.66      0.71        35
        Pass       0.88      0.93      0.90        95

    accuracy                           0.85       130
   macro avg       0.82      0.79      0.81       130
weighted avg       0.85      0.85      0.85       130

Classification Report on G2:
               precision    recall  f1-score   support

        Fail       0.67      0.59      0.62   

As you can see, the neural network model isn't a significant improvement over the previous model. My hypothesis here is that the data we are providing isn't containing all the important features. Let's now use the data that also has G1 and G2 as features.

## AdaBoost model on new data

In [27]:
param_grid = {
    'model__n_estimators': np.arange(4, 128, 4),
    'model__learning_rate': np.arange(0.05, 1.0, 0.05),
    'model__estimator__max_depth': np.arange(1, 10, 1),
    'model__estimator__min_samples_leaf': np.arange(1, 10, 1),
}

grid_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, n_iter=1024, scoring='accuracy', n_jobs=-1, verbose=1, random_state=2137) 

bestModel = grid_search.fit(G3_X_with_G1_G2_train, G3_y_with_G1_G2_train).best_estimator_

best_params = grid_search.best_params_
best_score = grid_search.best_score_

G3_pred = bestModel.predict(G3_X_with_G1_G2_test)

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
Best Parameters: {'model__n_estimators': 48, 'model__learning_rate': 0.15000000000000002, 'model__estimator__min_samples_leaf': 2, 'model__estimator__max_depth': 8}
Best Score: 0.9557505601194922


## AdaBoost model metrics

In [28]:
accuracy_G3 = accuracy_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Accuracy on G3:", accuracy_G3)

precision_G3 = precision_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Precision on G3:", precision_G3)

recall_G3 = recall_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Recall on G3:", recall_G3)

f1_G3 = f1_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("F1 Score on G3:", f1_G3)

conf_matrix_G3 = confusion_matrix(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Confusion Matrix on G3:\n", conf_matrix_G3)

target_names = ['Fail', 'Pass']
classification_report_G3 = classification_report(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred, target_names=target_names)
print("Classification Report on G3:\n", classification_report_G3) 

Accuracy on G3: 0.8923076923076924
Precision on G3: 0.9181818181818182
Recall on G3: 0.9528301886792453
F1 Score on G3: 0.9351851851851852
Confusion Matrix on G3:
 [[ 15   9]
 [  5 101]]
Classification Report on G3:
               precision    recall  f1-score   support

        Fail       0.75      0.62      0.68        24
        Pass       0.92      0.95      0.94       106

    accuracy                           0.89       130
   macro avg       0.83      0.79      0.81       130
weighted avg       0.89      0.89      0.89       130



This model hasn't improved much. Let's see the neural network.

## Neural network on the new data

In [29]:
from keras_tuner import RandomSearch
from keras import models, layers, callbacks

def build_model(hp):
    model = models.Sequential([
        layers.Input((32,)),
    ])

    units = hp.Int('units', min_value=32, max_value=128, step=32)
    current_units = units

    for i in range(1, hp.Int('hidden_layers', min_value=2, max_value=6, step=1)):

        model.add(layers.Dense(units=current_units, activation=hp.Choice('activation', ['relu', 'sigmoid', 'softmax'])))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
        current_units = int(current_units // hp.Float('size_factor', min_value=1.0, max_value=4.0, step=0.5))

    model.add(layers.Dense(1, activation='sigmoid',))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

early_stopping = callbacks.EarlyStopping(monitor='accuracy', patience=8) 

transformed_G3_X_with_G1_G2_train = Pipeline(steps=preprocessing_steps).fit_transform(G3_X_with_G1_G2_train, G3_y_with_G1_G2_train)
transformed_G3_X_with_G1_G2_test = Pipeline(steps=preprocessing_steps).fit_transform(G3_X_with_G1_G2_test, G3_y_with_G1_G2_test)

tuner = RandomSearch(hypermodel=build_model, objective='val_accuracy', max_trials=128, seed=2137, project_name='G3_with_G1_G2')
tuner.search(transformed_G3_X_with_G1_G2_train, G3_y_with_G1_G2_train, epochs=128, callbacks=[early_stopping], validation_data=(transformed_G3_X_with_G1_G2_test, G3_y_with_G1_G2_test), verbose=1)

model = tuner.get_best_models()[0]

print("Model summary:")
print(tuner.results_summary(1))

G3_pred_train = model.predict(transformed_G3_X_with_G1_G2_train)
G3_pred_train = np.where(G3_pred_train >= 0.5, 1, 0)

accuracy_G3 = accuracy_score(y_true=G3_y_with_G1_G2_train, y_pred=G3_pred_train)
print("Accuracy on train G3:", accuracy_G3)

G3_pred = model.predict(transformed_G3_X_with_G1_G2_test)
G3_pred = np.where(G3_pred >= 0.5, 1, 0)

Reloading Tuner from ./G3_with_G1_G2/tuner0.json
Model summary:
Results summary
Results in ./G3_with_G1_G2
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 004 summary
Hyperparameters:
units: 128
hidden_layers: 4
activation: relu
dropout: 0.1
size_factor: 4.0
Score: 0.9384615421295166
None
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Accuracy on train G3: 0.9980732177263969
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step


/Users/bigpoppe/UJ/PSI/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Model's metrics

In [30]:
print(model.summary())

accuracy_G3 = accuracy_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Accuracy on G3:", accuracy_G3)

precision_G3 = precision_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Precision on G3:", precision_G3)

recall_G3 = recall_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Recall on G3:", recall_G3)

f1_G3 = f1_score(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("F1 Score on G3:", f1_G3)

conf_matrix_G3 = confusion_matrix(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred)
print("Confusion Matrix on G3:\n", conf_matrix_G3)

target_names = ['Fail', 'Pass']
classification_report_G3 = classification_report(y_true=G3_y_with_G1_G2_test, y_pred=G3_pred, target_names=target_names)
print("Classification Report on G3:\n", classification_report_G3) 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,297 (36.32 KB)

 Trainable params: 8,961 (35.00 KB)

 Non-trainable params: 336 (1.31 KB)

None
Accuracy on G3: 0.9384615384615385
Precision on G3: 0.9537037037037037
Recall on G3: 0.9716981132075472
F1 Score on G3: 0.9626168224299065
Confusion Matrix on G3:
 [[ 19   5]
 [  3 103]]
Classification Report on G3:
               precision    recall  f1-score   support

        Fail       0.86      0.79      0.83        24
        Pass       0.95      0.97      0.96       106

    accuracy                           0.94       130
   macro avg       0.91      0.88      0.89       130
weighted avg       0.94      0.94      0.94       130

